<a href="https://colab.research.google.com/github/Ri796/Handwritten-digit-recognition/blob/main/Handwritten-digit-recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("PyTorch version:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


PyTorch version: 2.9.0+cu126
Device: cuda


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

# MNIST pixels(0-255)... mean is 33.31 and sd is 78.57
Transform= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

final_dataset= datasets.MNIST(root='data', train=True, download=True, transform=Transform)
print("size of the final_dataset",len(final_dataset))



#splitting the dataset in 3:2 ratio
total_size= len(final_dataset)
train_size= int(total_size*0.6)
test_size= total_size-train_size
train_dataset, test_dataset= random_split(final_dataset, [train_size, test_size])
print("size of the train_dataset",len(train_dataset))
print("size of the test_dataset",len(test_dataset))



batch_size=64
train_loader= DataLoader(train_dataset, batch_size, shuffle=True)
test_loader= DataLoader(test_dataset, batch_size, shuffle=False)

In [ ]:
batch_imgs, batch_labels = next(iter(train_loader))
print("Batch imgs shape:", batch_imgs.shape)   # [B, C, H, W]
print("Batch labels shape:", batch_labels.shape)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1=nn.Conv2d(1,32, kernel_size=3)
    self.conv2=nn.Conv2d(32,64, kernel_size=3)
    self.conv3=nn.Conv2d(64,128, kernel_size=3)
    self.pool=nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1=nn.Linear(128*1*1,128)
    self.fc2=nn.Linear(128,10)


  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=self.pool(x)
    x=F.relu(self.conv2(x))
    x=self.pool(x)
    x=F.relu(self.conv3(x))
    x=self.pool(x)
    x = x.view(x.size(0), -1)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return x

model=CNN().to(device)
print(model)

In [ ]:
import torch.optim as optim

loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def one_epoch(model, loader):
    model.train() #training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        #zero the parameter gradients
        optimizer.zero_grad()

        #Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        #Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        #Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_accuracy = (correct_predictions / total_samples) * 100

    print(f'Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.2f}%')
    return epoch_loss, epoch_accuracy

In [ ]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss, train_acc = one_epoch(model, train_loader)
    print(f"Epoch {epoch+1}:  Loss={train_loss:.4f},  Accuracy={train_acc:.4f}")


In [ ]:
torch.save(model.state_dict(), 'mnist_cnn_model.pth')

In [ ]:
def test_checker(model, loader):
    model.eval()  #evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            #Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)


            running_loss += loss.item() * images.size(0)

            #Calculate accuracy
            predicted = outputs.argmax(dim=1) # Removed .cpu().numpy()
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        test_loss = running_loss / total_samples
        test_accuracy = (correct_predictions / total_samples) * 100

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    return test_loss, test_accuracy

In [ ]:
import matplotlib.pyplot as plt
import torch

#Fetching one batch for visualization
dataiter = iter(test_loader)
images, labels = next(dataiter)

#Moving images and labels to the device(gpu)
images, labels = images.to(device), labels.to(device)

model.eval()
with torch.no_grad():
    outputs = model(images)
    _, preds = torch.max(outputs, 1)

plt.figure(figsize=(12,3))
for i in range(12):
    img_disp = images[i].squeeze().cpu() * 0.3081 + 0.1307  #denormalize
    plt.subplot(1,12,i+1)
    plt.imshow(img_disp, cmap='gray')
    plt.title(f"T:{labels[i].item()} \nP:{preds[i].item()}")
    plt.axis('off')
plt.show()

In [ ]:
test_loss, test_acc = test_checker(model, test_loader)
